# Spark RDD Command Note

**Outline**

* [Intro](#intro)
* [Spark RDD API](#rdd)

---

# <a id='intro'>Intro</a>

Spark has two different kinds of APIs

* **APIs**
    * RDD API: lower level, we should use this when we deal with unstructured data
    * DataFrame API: can be related to pandas dataframe in python.
        * SparkSQL    
* **2 modes**
    * shell
    * script
    
each block in the hadoop concept correspond to a partition of RDD.
One file correspond to a RDD.

* **Pros of Spark to MapReduce**: The main advantage of using Spark is that it can hold a portion of the original data in memory. It's easier to wrote any kinds of algorithms.

---

# <a id='rdd'>Spark RDD API</a>

Nothing is going to happen without an action

* **Transformations**: RDD to RDD
    * **map**: takes every elements of a partition in RDD and do something
    * **filter**:
    * **reduceByKey**: since it will reduce by key, it's a transformation
    * **flatMap**: one element with multiple output
    * **sortByKey**: Sort (key, value) RDD by key
    * **sortBy**: sort by custom condition or self select value
    * **distinct**: get distinct values in RDD
    * **groupBy**
    * **groupByKey**    
    * **mapValues**
    * **join**: inner join
        * **leftOuterJoin**: left join    
        * **rightOuterJoin**: right join    
        * **fullOuterJoin**: outer join  
    * **mapPartitions**: do map function for each partition. The input is all the records in each partition and the output is a single element
* **Actions**: RDD to various things
    * **collect**: take all the data into memory
    * **take**: return the first n elements of RDD
    * **first**: return the first element of RDD
    * **reduce**: reduce function without a key

* To take a

There are two ways to create RDD.
* Read from file
* create from scratch


> **Create RDD from scratch**

In [120]:
# the second parameter indicates the number of parititions in the RDD
myRDD = sc.parallelize([1,2,3,4,5], 2)

In [121]:
myRDD.getNumPartitions()

2

> **Get the number of records in each partition using mapParititions**

In [122]:
myRDD = sc.parallelize([1,2,3,4,5], 2)

In [123]:
def countRecord(x):
    count = 0
    for i in x:
        count = count + 1
    return [count]

In [124]:
myRDD.mapPartitions(countRecord).take(2)

[2, 3]

In [130]:
def isEven(x):
    """get all even number and output as list"""
    resultList = []
    for i in x:
        if i%2==0:
            resultList.append(i)
    return resultList

In [131]:
myRDD.mapPartitions(isEven).collect()

[2, 4]

> **Return vs Yield using mapPartitions**

In [6]:
myRDD = sc.parallelize([1,2,3,4,5],2)

In [7]:
myRDD.getNumPartitions()

2

In [9]:
# return: returns an object that we can be iterated as many time as we want
def customMapPartitions(x):
    res = []
    for i in x:
        res.append(i)
    return res

In [13]:
# yield: returns an object that can only be iterated once
def customMapPartitions2(x):
    res = []
    for i in x:
        res.append(i)
    yield res

In [11]:
myRDD.mapPartitions(customMapPartitions).first()

1

In [14]:
myRDD.mapPartitions(customMapPartitions2).first()

[1, 2]

> **broadcast variable in spark, similar to distributed cache in map reduce**

In [129]:
# # after running this, the variable is also been broadcasted in the memory of each server
toShare = sc.broadcst([1,2,3,4])
toShare.value

> **Read file**

In [6]:
myRDD2 = sc.textFile("data/Crimes_-_2001_to_present.csv")

In [7]:
myRDD2.take(2)

['ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location',
 '10078659,HY267429,05/19/2015 11:57:00 PM,010XX E 79TH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,true,false,0624,006,8,44,15,1184626,1852799,2015,05/26/2015 12:42:06 PM,41.751242944,-87.599004724,"(41.751242944, -87.599004724)"']

> **map (transformation)**

Each row will be mapped to one output 

* row1 -> list1
* row2 -> list2

In [8]:
myRDD.map(lambda x:x+1)
myRDD.collect()

[1, 2, 3, 4, 5]

In [9]:
def f(x):
    return x+1

In [10]:
myRDD.map(f).collect()

[2, 3, 4, 5, 6]

In [11]:
myRDD2.map(lambda x: x.split(",")).take(2)

[['ID',
  'Case Number',
  'Date',
  'Block',
  'IUCR',
  'Primary Type',
  'Description',
  'Location Description',
  'Arrest',
  'Domestic',
  'Beat',
  'District',
  'Ward',
  'Community Area',
  'FBI Code',
  'X Coordinate',
  'Y Coordinate',
  'Year',
  'Updated On',
  'Latitude',
  'Longitude',
  'Location'],
 ['10078659',
  'HY267429',
  '05/19/2015 11:57:00 PM',
  '010XX E 79TH ST',
  '143A',
  'WEAPONS VIOLATION',
  'UNLAWFUL POSS OF HANDGUN',
  'STREET',
  'true',
  'false',
  '0624',
  '006',
  '8',
  '44',
  '15',
  '1184626',
  '1852799',
  '2015',
  '05/26/2015 12:42:06 PM',
  '41.751242944',
  '-87.599004724',
  '"(41.751242944',
  ' -87.599004724)"']]

> **flatMap (transformation)**

Each element of row will be one output

Each row will be mapped to one output 

* element1 in row1 -> output1
* element2 in row1 -> output2
* element3 in row1 -> output3
...

In [132]:
rdd = sc.parallelize(['This is great great great'])

In [133]:
rdd.flatMap(lambda x: x.split(" ")).take(2)

['This', 'is']

In [136]:
rdd.flatMap(lambda x: x.split(" ")).map(lambda x: (x,1)).collect()

[('This', 1), ('is', 1), ('great', 1), ('great', 1), ('great', 1)]

> **SortBy multiple columns/values conditions**

* [StackOverFlow](https://stackoverflow.com/questions/36963319/spark-rdd-sort-by-two-values)
* [takeOrdered descending Pyspark](https://stackoverflow.com/questions/30787635/takeordered-descending-pyspark)

In [247]:
temp = sc.parallelize([['panda',(3,4)],['dog',(2,7)],['dog',(3,6)],['panda',(2,2)],['dog',(1,2)]], 2)

In [248]:
temp.collect()

[['panda', (3, 4)],
 ['dog', (2, 7)],
 ['dog', (3, 6)],
 ['panda', (2, 2)],
 ['dog', (1, 2)]]

In [249]:
# ascending order
temp.sortBy(lambda r: (r[0][0], int(r[1][0]))).collect()

[['dog', (1, 2)],
 ['dog', (2, 7)],
 ['dog', (3, 6)],
 ['panda', (2, 2)],
 ['panda', (3, 4)]]

In [250]:
# descending order of the second value
temp.sortBy(lambda r: (r[0][0], -int(r[1][0]))).collect()

[['dog', (3, 6)],
 ['dog', (2, 7)],
 ['dog', (1, 2)],
 ['panda', (3, 4)],
 ['panda', (2, 2)]]

> **reduce (action)**

In reduce function, one is current element and another is running sum

In [14]:
myRDD.reduce(lambda x,y: x+y)

15

> **Calculate average/mean for each key**

[StackOverFlow](https://stackoverflow.com/questions/40087483/spark-average-of-values-instead-of-sum-in-reducebykey-using-scala?noredirect=1&lq=1)

In [252]:
temp = sc.parallelize([['panda',10],['panda',8],['dog',9],['cat',5],['dog',12]], 2)

In [253]:
temp2 = temp.map(lambda x: (x[0],(x[1],1)))

In [254]:
temp2.collect()

[('panda', (10, 1)),
 ('panda', (8, 1)),
 ('dog', (9, 1)),
 ('cat', (5, 1)),
 ('dog', (12, 1))]

* x: current value, in our case, it's (1,1) or (2,1)...etc
* y: running sum. 

* x[0]: indicate the current value using to calculate the sum
* x[1]: indicate the current value using to calculate the total count

Therefore, in the key level for panda
* In the first iteration
    * x[0] = 10, x[1] = 1, y[0] = 0, y[1] = 0
* In the second iteration
    * x[0] = 8, x[1] = 1, y[0] = 10, y[1] = 1
* No more records
    * y[0] = 18, y[1] = 2


In [255]:
temp3 = temp2.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))

In [256]:
temp3.collect()

[('panda', (18, 2)), ('cat', (5, 1)), ('dog', (21, 2))]

In [257]:
temp4 = temp3.map(lambda r: (r[0], r[1][0]/r[1][1]))

In [258]:
temp4.collect()

[('panda', 9.0), ('cat', 5.0), ('dog', 10.5)]

In [259]:
# in one line
temp.map(lambda x: (x[0],(x[1],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda r: (r[0], r[1][0]/r[1][1])).collect()

[('panda', 9.0), ('cat', 5.0), ('dog', 10.5)]

> **groupByKey, mapValues: append values into a list**

In [26]:
temp = sc.parallelize([['panda',10],['panda',8],['dog',12],['cat',5],['dog',1]], 2)

In [27]:
temp.collect()

[['panda', 10], ['panda', 8], ['dog', 12], ['cat', 5], ['dog', 1]]

In [28]:
temp.groupByKey().mapValues(list).collect()

[('panda', [10, 8]), ('cat', [5]), ('dog', [12, 1])]

In [29]:
temp.sortBy(lambda x: x[1]).groupByKey().mapValues(list).collect()

[('cat', [5]), ('panda', [8, 10]), ('dog', [1, 12])]

In [32]:
# use may and reduceByKey. It looks more messy 
# https://stackoverflow.com/questions/27002161/reduce-a-key-value-pair-into-a-key-list-pair-with-apache-spark
temp.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda p,q: p+q).collect()

[('panda', [10, 8]), ('cat', [5]), ('dog', [12, 1])]

> **produce RDD[(X, X)] of all possible combinations from RDD[X], cartesian**

[StackOverFlow](https://stackoverflow.com/questions/26557873/spark-produce-rddx-x-of-all-possible-combinations-from-rddx)

In [22]:
temp = sc.parallelize([['panda',10],['lion',8],['dog',12],['cat',5],['bird',1]], 2)
temp.collect()

[['panda', 10], ['lion', 8], ['dog', 12], ['cat', 5], ['bird', 1]]

In [25]:
# it will still have duplicated rows. Figure it out later
temp.cartesian(temp).filter(lambda x: x[0]!=x[1]).collect()

[(['panda', 10], ['lion', 8]),
 (['lion', 8], ['panda', 10]),
 (['panda', 10], ['dog', 12]),
 (['panda', 10], ['cat', 5]),
 (['lion', 8], ['dog', 12]),
 (['lion', 8], ['cat', 5]),
 (['panda', 10], ['bird', 1]),
 (['lion', 8], ['bird', 1]),
 (['dog', 12], ['panda', 10]),
 (['dog', 12], ['lion', 8]),
 (['cat', 5], ['panda', 10]),
 (['cat', 5], ['lion', 8]),
 (['bird', 1], ['panda', 10]),
 (['bird', 1], ['lion', 8]),
 (['dog', 12], ['cat', 5]),
 (['cat', 5], ['dog', 12]),
 (['dog', 12], ['bird', 1]),
 (['cat', 5], ['bird', 1]),
 (['bird', 1], ['dog', 12]),
 (['bird', 1], ['cat', 5])]

> **join**

In [33]:
temp1 = sc.parallelize([['panda',10],['lion',8],['dog',12],['cat',5],['bird',1]], 2)
temp2 = sc.parallelize([['panda',10],['panda',8],['dog',12],['cat',5],['dog',1]], 2)

In [34]:
temp1.join(temp2).collect()

[('panda', (10, 10)),
 ('panda', (10, 8)),
 ('cat', (5, 5)),
 ('dog', (12, 12)),
 ('dog', (12, 1))]

In [48]:
temp3 = sc.parallelize([['panda',(10,'black')],['lion',(8,'yellow')],['dog',(12,'white')],['cat',(5,'grey')],['bird',(1,'green')]], 2)
temp4 = sc.parallelize([['panda',10],['dog',20],['horse',20]], 2)

In [49]:
# inner join
temp3.join(temp4).collect()
temp4.join(temp3).collect()

[('panda', (10, (10, 'black'))), ('dog', (20, (12, 'white')))]

In [50]:
# left join
temp3.leftOuterJoin(temp4).collect()

[('panda', ((10, 'black'), 10)),
 ('lion', ((8, 'yellow'), None)),
 ('cat', ((5, 'grey'), None)),
 ('bird', ((1, 'green'), None)),
 ('dog', ((12, 'white'), 20))]

In [51]:
# right join
temp3.rightOuterJoin(temp4).collect()

[('panda', ((10, 'black'), 10)),
 ('horse', (None, 20)),
 ('dog', ((12, 'white'), 20))]

In [52]:
# right join
temp3.fullOuterJoin(temp4).collect()

[('panda', ((10, 'black'), 10)),
 ('lion', ((8, 'yellow'), None)),
 ('cat', ((5, 'grey'), None)),
 ('bird', ((1, 'green'), None)),
 ('horse', (None, 20)),
 ('dog', ((12, 'white'), 20))]